In [1]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *


In [2]:
import os
os.environ['SPEDAS_DATA_DIR'] = f"{os.environ['HOME']}/data"

In [3]:
#| code-summary: import all the packages needed for the project
#| output: hide

from fastcore.utils import *
from fastcore.test import *


from ids_finder.utils import *
from ids_finder.core import *

import polars as pl
try:
    import modin.pandas as pd
    import modin.pandas as mpd
except ImportError:
    import pandas as pd

import pandas
import numpy as np
import xarray as xr


from datetime import timedelta
from loguru import logger
import speasy as spz
from multipledispatch import dispatch

import altair as alt


## Dataset Overview

In [4]:
stereo_probes = ["a", "b"]
probe = stereo_probes[0]

jno_start_date = "2011-08-25"
jno_end_date = "2016-06-30" 

trange = [jno_start_date, jno_end_date]
test_trange = ["2011-08-25", "2012-08-26"]

In [ ]:
sat = 'STA'
coord = 'rtn'

In [5]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.

2023-09-28 22:24:28.097 | INFO     | __main__:<module>:4 - STEREO Ahead IMPACT/MAG Magnetic Field Vectors (RTN) - J. Luhmann (UCB/SSL)
2023-09-28 22:24:28.097 | INFO     | __main__:<module>:5 - sta_l1_mag_rtn_cdaweb
2023-09-28 22:24:28.098 | INFO     | __main__:<module>:6 - Magnetic field vector in RTN coordinates from the IMPACT/MAG instrument.
2023-09-28 22:24:28.098 | INFO     | __main__:<module>:7 - STA_L1_MAG_RTN/BFIELD


'Magnetic field vector in RTN coordinates from the IMPACT/MAG instrument.'

### Download all the files

Download data using `pyspedas`, but load it using `pycdfpp` (using `pyspedas` to load the data directly into `xarray` is very slow)

In [6]:
import pyspedas
import pycdfpp

In [7]:
files = pyspedas.stereo.mag(trange, downloadonly=True)
cdfs = [pycdfpp.load(file) for file in files]
times = [pycdfpp.to_datetime64(cdf["Epoch"]) for cdf in cdfs]
BFIELDs = [cdf["BFIELD"].values for cdf in cdfs]

time = np.concatenate(times)
BFIELD = np.concatenate(BFIELDs)

df = pl.DataFrame({
    "time": time,
    "BX": BFIELD[:,0],
    "BY": BFIELD[:,1],
    "BZ": BFIELD[:,2],
    "B": BFIELD[:,3],
})

28-Sep-23 22:25:07: Downloading remote index: http://sprg.ssl.berkeley.edu/data/misc/stereo/impact/level1/ahead/mag/RTN/2011/08/
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110825_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110826_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110827_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110828_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110829_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110830_V06.cdf
28-Sep-23 22:25:07: File is current: /home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2

OSError: [Errno 28] No space left on device: '/tmp/tmpthwqpzpo' -> '/home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2013/01/STA_L1_MAG_RTN_20130104_V06.cdf'

In [55]:
output = f"../data/sta_fgm.parquet"
vars = ['BFIELD']

def convert_cdf_to_parquet(files, vars, output):
    # if os.path.exists(output):
    #     return output

    vars_dict = {var: [] for var in vars}
    times = []
    for file in files:
        cdf = pycdfpp.load(file)
        time = pycdfpp.to_datetime64(cdf["Epoch"])
        for var in vars:
            vars_dict[var].append(cdf[var].values)
        times.append(time)
    
    for var in vars:
        vars_dict[var] = np.concatenate(vars_dict[var])
    times = np.concatenate(times)
    df = pl.DataFrame({**vars_dict, "time": times})
    # df.write_parquet(output)
    return output
    
    
convert_cdf_to_parquet(files[0:1], vars, output)

BFIELD
list[f32]
"[-3.231869, 2.525943, … 4.511669]"
"[-3.260727, 2.588474, … 4.552812]"
"[-3.231348, 2.60437, … 4.535828]"
"[-3.231398, 2.590189, … 4.527003]"
"[-3.231665, 2.582672, … 4.546539]"
"[-3.231548, 2.598732, … 4.55039]"
"[-3.231429, 2.614794, … 4.554329]"
"[-3.231281, 2.606249, … 4.531057]"
"[-3.216715, 2.604288, … 4.525378]"


In [45]:
pl.read_parquet(output)

BFIELD,time
list[f32],datetime[ns]
"[-3.231869, 2.525943, … 4.511669]",2011-08-25 00:00:01.868
"[-3.260727, 2.588474, … 4.552812]",2011-08-25 00:00:01.993
"[-3.231348, 2.60437, … 4.535828]",2011-08-25 00:00:02.118
"[-3.231398, 2.590189, … 4.527003]",2011-08-25 00:00:02.243
"[-3.231665, 2.582672, … 4.546539]",2011-08-25 00:00:02.368
"[-3.231548, 2.598732, … 4.55039]",2011-08-25 00:00:02.493
"[-3.231429, 2.614794, … 4.554329]",2011-08-25 00:00:02.618
"[-3.231281, 2.606249, … 4.531057]",2011-08-25 00:00:02.743
"[-3.216715, 2.604288, … 4.525378]",2011-08-25 00:00:02.868


In [41]:
files[0:1]

['/home/zzhang/data/stereo/impact/level1/ahead/mag/RTN/2011/08/STA_L1_MAG_RTN_20110825_V06.cdf']

In [32]:
# cdf = pycdfpp.load(file)
cdf['BFIELD'].values
# cdf['Epoch'].values

array([(6.34814496e+13,), (6.34814496e+13,), (6.34814496e+13,), ...,
       (6.34815360e+13,), (6.34815360e+13,), (6.34815360e+13,)],
      dtype=[('value', '<f8')])

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [15]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)


Download data in a background thread

In [9]:
@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")

In [16]:
%%markdown
download_data(products, trange)

download_data(products, trange)


28-Sep-23 10:53:55: Using pycdfpp


### Convert data to `parquet` for faster processing

In [17]:
def spz2parquet(data, force=False):
    output = f"../data/{data.name}.parquet"
    if Path(output).exists() and not force:
        logger.info("Data already converted to parquet")
    else: 
        df = pandas.DataFrame(
            data.values, index=pandas.Series(data.time, name="time"), columns=data.columns
        )
        
        df.to_parquet(output)
        logger.info("Data converted to parquet successfully")

### Check and preprocess the data

As we are only interested in the data when THEMIS is in the solar wind, for simplicity we will only keep the data when `X, SSE` and `X, GSE` is positive.

- State data time resolution is 1 minute...

- FGS data time resolution is 4 second...

In [ ]:
# get_time_dff(sat_state)
# get_time_dff(data)

In [ ]:
sat = "sta"
files = f"../data/{sat}_{datatype}_{coord}.parquet"
output = f"../data/{sat}_data.parquet"

rename_mapping = {
    "Bx FGS-D": "BX",
    "By FGS-D": "BY",
    "Bz FGS-D": "BZ",
}

if Path(output).exists():
    pass
else:
    data = pl.scan_parquet(files).rename(rename_mapping).unique("time").sort("time")
    data_sw = data.join_asof(sat_state_sw, on="time", tolerance="1m").drop_nulls().collect()
    data_sw.write_parquet(output)

In [ ]:
%%markdown
df = (
    sat_state_sw.upsample("time", every="1m")
    .group_by_dynamic("time", every="1d")
    .agg(pl.col("X, SSE").null_count().alias("null_count"))
    .with_columns(
        pl.when(pl.col("null_count") > 720).then(0).otherwise(1).alias("availablity")
    )
)

properties = {
    'width': 800,
}

chart1 = alt.Chart(df).mark_point().encode(
    x='time',
    y='null_count'
).properties(**properties)

chart2  = alt.Chart(df).mark_point().encode(
    x='time',
    y='availablity'
).properties(**properties)

(chart1 & chart2)

df = (
    sat_state_sw.upsample("time", every="1m")
    .group_by_dynamic("time", every="1d")
    .agg(pl.col("X, SSE").null_count().alias("null_count"))
    .with_columns(
        pl.when(pl.col("null_count") > 720).then(0).otherwise(1).alias("availablity")
    )
)

properties = {
    'width': 800,
}

chart1 = alt.Chart(df).mark_point().encode(
    x='time',
    y='null_count'
).properties(**properties)

chart2  = alt.Chart(df).mark_point().encode(
    x='time',
    y='availablity'
).properties(**properties)

(chart1 & chart2)


## Processing the whole data

In [ ]:
def get_memory_usage(data):
    datatype = type(data)
    match datatype:
        case pl.DataFrame:
            size = data.estimated_size()
        case pd.DataFrame:
            size = data.memory_usage().sum()
        case xr.DataArray:
            size = data.nbytes

    logger.info(f"{naturalsize(size)} ({datatype.__name__})")
    return size

In [ ]:
#| eval: false
tau = timedelta(seconds=60)
data_resolution = timedelta(seconds=4)
files = f"../data/{sat}_data_sw.parquet"
output = f'../data/{sat}_candidates_sw_tau_{tau.seconds}.parquet'

data = pl.scan_parquet(files).set_sorted('time').collect()
sat_fgm = df2ts(
    data, ["BX", "BY", "BZ"], attrs={"coordinate_system": coord, "units": "nT"}
)
get_memory_usage(data)
get_memory_usage(sat_fgm)

indices = compute_indices(data, tau)

# filter condition
sparse_num = tau / data_resolution // 3
filter_condition = get_ID_filter_condition(sparse_num = sparse_num)

candidates_pl = indices.filter(filter_condition).with_columns(pl_format_time(tau))
candidates = convert_to_dataframe(candidates_pl)
get_memory_usage(candidates)
# del indices

2023-09-27 11:57:07.031 | INFO     | __main__:get_memory_usage:11 - 741.8 MB (DataFrame)
2023-09-27 11:57:07.031 | INFO     | __main__:get_memory_usage:11 - 222.6 MB (DataArray)
27-Sep-23 11:57:09: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


27-Sep-23 11:57:11: Unable to poll TPU GCE metadata: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/attributes/accelerator-type (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
27-Sep-23 11:57:11: Failed to detect number of TPUs: [Errno 2] No such file or directory: '/dev/vfio'
2023-09-27 11:57:12,367	INFO worker.py:1642 -- Started a local Ray instance.
27-Sep-23 11:57:13: UserWarning: Distributing <clas

6335828

In [ ]:
#| eval: false
ids = process_candidates(candidates, sat_fgm, data, data_resolution)
ids = ids.unique(["d_time", "d_tstart", "d_tstop"])
ids.write_parquet(output)

Estimated completion of line 17: 100%██████████ Elapsed time: 00:00, estimated remaining time: 00:00


In [ ]:
#| eval: false

test_eq(ids.unique(["d_time", "d_tstart", "d_tstop"]).shape, ids.unique("d_time").shape)

## Obsolete codes

In [ ]:
#| eval: false
import pycdfpp
import pyspedas

In [ ]:
#| eval: false

def convert_thm_state_to_parquet(
    probe: str, trange
):
    file_name = f"./data/th{probe}_state.parquet"
    if os.path.exists(file_name):
        return file_name

    start = trange.start.to_string()
    end = trange.end.to_string()

    files = pyspedas.themis.state(
        probe=probe,
        trange=[start, end],
        downloadonly=True,
        no_update=True,
    )

    thm_pos_sse_Xs = []
    thm_pos_gse_Xs = []
    thm_state_times = []
    for file in files:
        thm_state = pycdfpp.load(file)
        epoch_dt64 = thm_state[
            f"time"
        ].values  #  CATDESC: "thm_state_time, UTC, in seconds since 01-Jan-1970 00:00:00"
        thm_pos_sse_Xs.append(thm_state[f"th{probe}_pos_sse"].values[:, 0])
        thm_pos_gse_Xs.append(thm_state[f"th{probe}_pos_gse"].values[:, 0])
        thm_state_times.append(epoch_dt64)

    thm_pos_sse_X = np.concatenate(thm_pos_sse_Xs)
    thm_pos_gse_X = np.concatenate(thm_pos_gse_Xs)
    thm_state_time = np.concatenate(thm_state_times)

    pl.DataFrame(
        {
            "thm_state_time": thm_state_time,
            "thm_pos_gse_X": thm_pos_gse_X,
            "thm_pos_sse_X": thm_pos_sse_X,
        }
    ).with_columns(
        pl.from_epoch(pl.col("thm_state_time"), time_unit="s")
    ).write_parquet(
        file_name
    )

    return file_name


def convert_thm_fgm_to_parquet(probe, trange):
    file_name = f"./data/th{probe}_fgm.parquet"
    if os.path.exists(file_name):
        return file_name

    start = trange.start.to_string()
    end = trange.end.to_string()
    
    files = pyspedas.themis.fgm(
        probe=probe,
        trange=[start, end],
        downloadonly=True,
        no_update=True,
    )

    thm_fgl_gses = []
    thm_fgl_btotals = []
    thm_fgl_times = []

    for file in files:
        cdf = pycdfpp.load(file)
        thm_fgl_gses.append(cdf[f"th{probe}_fgl_gse"].values)
        thm_fgl_btotals.append(cdf[f"th{probe}_fgl_btotal"].values)
        thm_fgl_times.append(cdf[f"th{probe}_fgl_time"].values)

    thm_fgl_gse = np.concatenate(thm_fgl_gses)
    thm_fgl_btotal = np.concatenate(thm_fgl_btotals)
    thm_fgl_time = np.concatenate(thm_fgl_times)

    pl.DataFrame(
        {
            "time": thm_fgl_time,
            "BX": thm_fgl_gse[:,0],
            "BY": thm_fgl_gse[:,1],
            "BZ": thm_fgl_gse[:,2],
            "B": thm_fgl_btotal,
        }
    ).with_columns(
        pl.from_epoch(pl.col("thm_fgl_time"), time_unit="s"),
    ).write_parquet(   
        file_name
    )
    
    return file_name

In [ ]:
#| eval: false
convert_thm_state_to_parquet(probe, trange)
convert_thm_fgm_to_parquet(probe, trange)